# 1. Executive Summary
This analysis uses ranking system to calculate revenue distribution and general performance on different sections.
The findings indicate that revenue is well distributed among countries, with the top-performer showing 18.6% share of total revenue, suggesting no overdependency on a single country to carry on the whole operation.
The same applies for products, no outlier detected with all products contributing considerably to total revenue. Although the top-performer has a gap of $30 million to the last one, the two highest on revenue summed only incorporate 11% of total revenue. It indicates moderate balance with few adjustements in order to increase specific products participation, no radical interference needed.
Sales representative performance present similar result, but with negative outliers. The two top-performers have incredible numbers, but are followed close by their colleagues. Some show rather ability in handling high-end products, and others on large volume. Talents to polish overtime and expect great contributions to the operation.

# 2. Business Overview
The objective of this analysis is to evaluate the overall performance of a chocolate sales operation. It focuses on identifying revenue contributions across countries, products, and sales representatives, as well as assessing revenue distribution, trend periods, personal-level performance, and future market opportunities.
* Check if there's a overly dependency on a group or individual country, product, or sales representative.
* Looks out for new opportunities such as sales representative dealing well with high-end products but low volume, country with huge volume but low average price per transaction.
* Support strategic by using data to create information, thus mitigating risks and data-basing key decisions for future operations and expansions.

With this understanding, board and stakeholders can determine current company health and whether change or not long-term plans.

# 3. Dataset Overview
The dataset contains 3282 transactional records spanning from 03/01/2022 to 31/08/2024 (not including months: 09, 10, 11, 12), covering 6 countries, 25 sales representatives, and 22 products.
- Source: Kaggle Datasets
- Rows: 3283
- Columns:
    - Date - Transaction date (DD/MM/YYYY Format)
    - Product - Full product name
    - Amount - Sales amount
    - Boxes - Quantity of boxes in each transaction
    - Country  - Customer’s country
    - Sales Person - name of the sales representative

# 4. Data Preparation and Cleaning
Creating the environment and loading key libraries (Pandas and Numpy).

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
print('Sucessful load')

/kaggle/input/chocolate-sales/ChocolateSales.pbix
/kaggle/input/chocolate-sales/Chocolate Sales (2).csv
/kaggle/input/chocolate-sales/ChocolateSales.pdf
/kaggle/input/chocolate-sales/ChocolateSales.pbit
/kaggle/input/datasets/josgomespinheironeto/chocolate-sales-new/Chocolate Sales (2) new.csv
Sucessful load


## 4.1 - Data Inspection
Initial inspection was conducted using Microsoft Excel to visually assess the dataset structure and identify potential data quality issues such as formatting inconsistencies, missing values, and duplicate records. First task was to properly format the data since it's a CSV file and opens on Excel with all data glued on the first column. For this, I used the Import Text Wizard, tool responsible for reading data in other formats, including `.csv`, and transforming it to Excel formats, as `.xls` or `.xlsx`.

* Column `Sales Person` has data type of string/text.
* Column `Country` has data type of string/text.
* Column `Product` has data type of string/text.
* Column `Date` has data type of string/text. Necessary change to datetime data type.
* Column `Amount` has data type of string/text. Necessary change to integer data type.
* Column `Boxes Shipped` has data type of integer.

No missing values, spelling errors, or duplicated values found in a preliminary Excel analysis.

## 4.2 - Data Validation
Although the dataset size allowed a manual inspection, programmatic validation via Pandas was conducted to ensure data quality and analysis reliability.

In [2]:
# Load dataset
db = pd.read_csv('/kaggle/input/chocolate-sales/Chocolate Sales (2).csv')

#Dataset overview
db.info()

#Check for duplicated values
db.duplicated().sum()

#Check for null values
db.isnull().sum()

# Validate numeric columns
db[['Amount', 'Boxes Shipped']].describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3282 entries, 0 to 3281
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Sales Person   3282 non-null   object
 1   Country        3282 non-null   object
 2   Product        3282 non-null   object
 3   Date           3282 non-null   object
 4   Amount         3282 non-null   object
 5   Boxes Shipped  3282 non-null   int64 
dtypes: int64(1), object(5)
memory usage: 154.0+ KB


,Boxes Shipped
count,3282.000000
mean,164.666971
std,124.024736
min,1.000000
25%,71.000000
50%,137.000000
75%,232.000000
max,778.000000


All records were successfully validated, with no missing values or structural inconsistencies detected. This adds an extra security layer before proceeding to data transformation and KPIs.

## 4.3 - Transformations

### 4.3.1 Date Standardization
The `Date` column will be converted to datetime format to enable time-based aggregation and chronological analysis. This transformation allows fully use of datetime specification such as extract specific month, day, or year, and prevent string-base errors.

In [3]:
# Update column with new data type
db['Date'] = pd.to_datetime(db['Date'], dayfirst=True)

### 4.3.2 Amount Numeric Transformation
The `Amount` column data type will be transformed from object(string) to int64(integer). This transformation is going to allow aggregation, KPI computation, and descriptive statistics without casting errors.

In [4]:
#Remove non-numeric character from Amount to change it from str to int
def remove(cell):
    return cell.replace('$', '').replace(',', '')

# Turn into numeric
db['Amount'] = pd.to_numeric(db['Amount'].map(remove))

### 4.3.3 Calculated Metrics
A derived metric, Revenue per Box, was calculated to assess pricing consistency and operational efficiency across products, countries, and sales representatives. Its creation enables deeper performance evaluation.

In [5]:
# Function to calculate revenue per box
def calculate(row):
    return row['Amount'] / row['Boxes Shipped']

# Read every row and store new Series
new_column = db.apply(calculate, axis='columns')

# Round and assign it
db['Revenue per Box'] = new_column.round(0)

### 4.3.4 Creation of Time-Based Feature
This additional column will be derived from the Date column to support aggregated performance analysis and trend evaluation. It allows deeper analysis without interfering with raw and primordial data, such as `Date` column.

In [6]:
# Extract each transacation month and assign to a new column
db['Month'] = db['Date'].dt.month
db.to_csv('choco_sales.csv', index=False)

# 5. Key Performance Indicators (KPIs)
In this section, we're going turn data into information by using SQL queries. 

## 5.1 Total Revenue
Total revenue represents the overall sales performance of the company across all countries, products, and sales representatives.

In [7]:
query = """SELECT SUM(Amount) AS Total_Revenue
            FROM choco_sales;
        """

| Total_Revenue |
| --- |
| 1303402410 |


Total revenue serves as the primary performance indicator and estabilishes a baseline for all future comparisons and inferences.

This dataset contains 3282 transactions generating a total revenue of $1,303,402,410 Dollars.

## 5.2 Revenue by Country
Revenue aggregated by country measures geographic market performance.

In [8]:
query = """SELECT country, SUM("Boxes Shipped") AS Total_boxes, SUM(Amount) AS Revenue_per_Country,
           SUM(Amount)/SUM("Boxes Shipped") AS Avg_price_per_box
           FROM choco_sales
           GROUP BY country
		   ORDER BY Revenue_per_Country DESC, Avg_price_per_box DESC;
        """

| Country      | Total_boxes | Revenue_per_Country | Avg_price_per_box |
|-------------|------------|--------------------|-------------------|
| Australia   | 99618      | 243278733          | 2442              |
| USA         | 81820      | 220385614          | 2693              |
| UK          | 92523      | 219184578          | 2368              |
| India       | 89968      | 216005965          | 2400              |
| Canada      | 95158      | 206129450          | 2166              |
| New Zealand | 81350      | 198418070          | 2439              |

This KPI identifies high-performant market and areas with potential growth for either high-end or volume-based products.

The top contributor present a market share of approxiamtely 19%. Despite the significative amount, the overall balance is well maintained with no outliers.

## 5.3 Revenue by Sales Representative
Revenue contribution by individual sales representatives.

In [9]:
query = """SELECT "Sales Person", SUM(Amount) AS Revenue_per_Sales_Person, SUM("Boxes Shipped") AS Total_boxes,
           SUM(Amount)/SUM("Boxes Shipped") AS Avg_price_per_box
           FROM choco_sales
           GROUP BY "Sales Person"
		   ORDER BY Revenue_per_Sales_Person DESC, Avg_price_per_box DESC;
        """

| Sales Person           | Revenue_per_Sales_Person | Total_boxes | Avg_price_per_box |
|------------------------|--------------------------|------------|-------------------|
| Kelci Walkden          | 69371340                 | 26605      | 2607              |
| Brien Boise            | 69289878                 | 24738      | 2800              |
| Madelene Upcott        | 68195064                 | 22199      | 3071              |
| Ches Bonnell           | 66934753                 | 23070      | 2901              |
| Van Tuxwell            | 66078076                 | 20627      | 3203              |
| Oby Sorrel             | 65921004                 | 26390      | 2497              |
| Dennison Crosswaite    | 62715934                 | 26862      | 2334              |
| Beverie Moffet         | 59928174                 | 28027      | 2138              |
| Barr Faughny           | 57552319                 | 19520      | 2948              |
| Marney O'Breen         | 57130545                 | 24595      | 2322              |
| Roddy Speechley        | 51456808                 | 21130      | 2435              |
| Kaine Padly            | 51045612                 | 22134      | 2306              |
| Gunar Cockshoot        | 50690897                 | 20299      | 2497              |
| Curtice Advani         | 46968737                 | 21599      | 2174              |
| Jan Morforth           | 46798479                 | 23360      | 2003              |
| Karlen McCaffrey       | 46553830                 | 29553      | 1575              |
| Jehu Rudeforth         | 45904105                 | 22104      | 2076              |
| Gigi Bohling           | 44200152                 | 19237      | 2297              |
| Mallorie Waber         | 43479543                 | 18219      | 2386              |
| Andria Kimpton         | 42899488                 | 19730      | 2174              |
| Camilla Castle         | 42426831                 | 16505      | 2570              |
| Dotty Strutley         | 40790499                 | 20927      | 1949              |
| Husein Augar           | 40274267                 | 17683      | 2277              |
| Rafaelita Blaksland    | 37767585                 | 13091      | 2885              |
| Wilone O'Kielt         | 29028490                 | 12233      | 2372              |


Measures individual performance, contribution to total revenue, and potential talents for specific groups of products.

The two top-performing sales representatives accounts for 11% of total revenue. The approximate $30 million gap between highest and lowest in revenue, shows an outlier for the last-placed sales representative. Despite that, this difference is noticeable for the last from the second-to-last performers, other positions have no such gap and follows a smooth decrescent order.

## 5.4 Revenue by Product
Revenue contribution by individual products.

In [10]:
query = """SELECT Product, SUM("Boxes Shipped") AS Total_boxes, SUM(Amount) AS Revenue_per_Product,
           SUM(Amount)/SUM("Boxes Shipped") AS Avg_price_per_box
           FROM choco_sales
           GROUP BY Product
		   ORDER BY Revenue_per_Product DESC, Avg_price_per_box DESC;
        """

| Product                 | Total_boxes | Revenue_per_Product | Avg_price_per_box |
|--------------------------|------------|---------------------|-------------------|
| Smooth Sliky Salty       | 26969      | 73619017            | 2729              |
| 50% Dark Bites           | 29810      | 72837584            | 2443              |
| Peanut Butter Cubes      | 25339      | 69795637            | 2754              |
| White Choc               | 25158      | 67353846            | 2677              |
| Eclairs                  | 26678      | 65928209            | 2471              |
| 85% Dark Bars            | 23828      | 62972528            | 2642              |
| Organic Choco Syrup      | 23602      | 61259575            | 2595              |
| Spicy Special Slims      | 26662      | 61047436            | 2289              |
| Manuka Honey Choco       | 23736      | 60521659            | 2549              |
| Mint Chip Choco          | 25149      | 60016855            | 2386              |
| 99% Dark & Pure          | 24818      | 59596797            | 2401              |
| Almond Choco             | 20558      | 57781283            | 2810              |
| After Nines              | 25156      | 57301671            | 2277              |
| Drinking Coco            | 26402      | 56930678            | 2156              |
| Raspberry Choco          | 21672      | 56463233            | 2605              |
| Milk Bars                | 25436      | 56086861            | 2205              |
| Fruit & Nut Bars         | 23632      | 55320216            | 2340              |
| Baker's Choco Chips      | 21448      | 54549285            | 2543              |
| Orange Choco             | 23607      | 54511459            | 2309              |
| Choco Coated Almonds     | 19677      | 50620121            | 2572              |
| Caramel Stuffed Bars     | 26576      | 46122079            | 1735              |
| 70% Dark Bites           | 24524      | 42766381            | 1743              |


Measures products' revenue contributions, volume of shippings, average pricing, and potential key products to be explored.

The two top-performes accounts for 11% of total revenue, showing no dominance or over-dependency. There's no indication of concentration in any perspective, suggesting stable performance, diversity of high-performing products, and success to reach target audiences.

## 5.5 Average Revenue per Transaction
Average transaction value across all records.

In [11]:
query = """SELECT AVG("Revenue per Box") AS Average_per_Box
            FROM choco_sales;
        """

| Average_per_Box |
| --- |
| 1113.40950639854 |

Indicates overall purchasing behavior and sales consistency across time and conditions.

The average price remains stable, ensuring the absence of extreme volatility.

## 5.6 Monthly Revenue
Volume of transactions in each month and total revenue in span time.

In [12]:
query = """SELECT Month, SUM("Boxes Shipped") AS Total_boxes, SUM(Amount) AS Revenue_per_Month,
           SUM(Amount)/SUM("Boxes Shipped") AS Avg_price_per_box
           FROM choco_sales
           GROUP BY Month
		   ORDER BY Revenue_per_Month DESC, Avg_price_per_box;
        """

| Month | Total_boxes | Revenue_per_Month | Avg_price_per_box |
|-------|------------|-------------------|-------------------|
| 1     | 84162      | 189298173         | 2249              |
| 6     | 80357      | 181271250         | 2255              |
| 7     | 69808      | 170393846         | 2440              |
| 3     | 59633      | 159835390         | 2680              |
| 5     | 66662      | 159225336         | 2388              |
| 8     | 60682      | 149989744         | 2471              |
| 2     | 54917      | 149913303         | 2729              |
| 4     | 64216      | 143475368         | 2234              |

Main indicators of seasonality and audience behaviour.

The two leading months are a generous surprise, and the closely following are to be expected given Easter holiday. No real concentration indicated. It's expected with such a classic product.

# 6 - Performance Analysis
## 6.1 Country Performance
**Australia** leads total revenue, representing approximately **18.6%** of overall sales. While it holds a clear first position, revenue distribution across the six countries remains relatively balanced, with no extreme market dependency observed. The **USA** and **UK** follow closely, forming a second performance tier alongside **Canada** and **India**.

**New Zealand** ranks slightly lower; however, the difference between positions is not substantial, reinforcing the diversified nature of the revenue structure.

When analyzing volume (**Boxes Shipped**) alongside average price per box, Australia’s leadership is primarily volume-driven. In contrast, the **USA** maintains competitive revenue despite lower shipment volume due to a higher average price per box. This indicates stronger pricing power or higher-value product mix.

**Canada** presents a growth opportunity: shipment volume is relatively strong, and a moderate increase in average price could significantly improve total revenue without requiring demand expansion.

## 6.2 Salesperson Performance
**Kelci Walkden** and **Brien Boise** are the top-performing sales representatives, generating the highest total revenue with minimal difference between them. **Madelene Upcott** follows closely, forming a strong leading group.

Revenue distribution across the team is generally stable, with moderate gaps between mid-level performers. However, the two lowest-performing representatives display a more pronounced revenue gap, indicating potential performance inefficiencies or territory-related challenges.

Volume analysis shows that Kelci Walkden’s leadership is supported by consistently high shipment numbers. B**everie Moffet** and **Karlen McCaffrey** demonstrate strong volume performance as well, suggesting potential for upward revenue mobility if average pricing improves.

Conversely, **Madelene Upcott** and **Van Tuxwell** stand out due to higher average price per box, indicating strength in closing higher-margin transactions.

This segmentation suggests two distinct sales strategies within the team:

* Volume-driven performers

* Margin-driven performers

Balancing both approaches may optimize total team revenue.

## 6.3 Product Performance
**Smooth Silky Salty** and **50% Dark Bites** are the top revenue-generating products, both significantly ahead of the third-ranked product. Together, they account for approximately **11%** of total revenue, which is notable given the overall balanced product distribution.

**Smooth Silky Salty** achieves its leadership through a combination of high shipment volume and strong average price positioning. In contrast, **50% Dark Bites** relies heavily on volume, maintaining a competitive but more moderate average price.

**Almond Choco** presents an interesting margin-driven opportunity. Despite lower shipment volume, it has the highest average price per box, indicating premium positioning.

Conversely, **Caramel Stuffed Bars** exhibits strong volume but the lowest average price. This product may benefit from price optimization strategies, as even minor pricing adjustments could significantly impact total revenue.

Overall, product performance reflects a mix of:

* High-volume core products

* Premium niche products

* Volume-heavy low-margin items

This diversified structure reduces product-level revenue concentration risk.

## 6.4 Revenue Trends Over Time
**January** and **June** generate the highest monthly revenues, followed by **July**. Although these months lead in total revenue, the overall monthly distribution remains relatively balanced without extreme seasonal concentration.

Revenue peaks in **January** and **June** are primarily volume-driven rather than price-driven, as these months exhibit some of the lowest average prices per box alongside the highest shipment volumes. This suggests demand expansion rather than premium pricing.

**February** and **March** display the highest average prices, while shipment volume is comparatively lower. This indicates potential demand compression during higher pricing periods.

While specific seasonal drivers cannot be confirmed without external data, the observed pattern suggests that volume-based growth periods outperform price-based strategies in this dataset.

# 7. Key Insights
* **Australia** leads country performance, accounting for approximately **18.6%** of total revenue. Despite its leadership, revenue distribution across countries remains **relatively balanced**, indicating **diversified geographic** exposure.
* **Canada** demonstrates strong **growth potential**, combining high shipment volume with moderate pricing. Even small pricing optimizations could significantly increase its **revenue contribution**.
* **Kelci Walkden** and **Brien Boise** are the most consistent sales performers, balancing **shipment volume** and **pricing efficiency** to maintain top positions.
* **Madelene Upcott** and **Van Tuxwell** stand out for **margin-driven performance**, achieving high average revenue per box and demonstrating strength in **premium** or **high-value transactions**.
* **Beverie Moffet** and **Karlen McCaffrey** show strong **volume-driven performance**, indicating **operational efficiency** and **demand generation** capacity.
* Product leadership is driven by distinct strategies:
    * **Smooth Silky Salty** combines **high volume** with **strong pricing**.
    * **50% Dark Bites** achieves top performance primarily through exceptional **shipment volume**.
* Monthly revenue peaks in **January** and **June**, largely driven by higher **shipment volume** rather than pricing increases.
* **February** and **March** record the highest average prices, although this is accompanied by lower shipment volume. While seasonal effects may influence this pattern, further data would be required for confirmation.

# 8. Strategic Recommendations
## 8.1 Optimize Price in High-Volume Markets
Canada and other countries have a steady shippment performance. An experiment on prices, such as minor price increases on high-demand products, could increase revenue without requiring sudden physical expansion or demand growth.

**Rationale:**
High volume with moderate pricing suggest a significant revenue expansion potential.

## 8.2 Leverage Margin-Driven Sales Representatives
Sales representatives such as Madelene Upcott and Van Tuxwell show strong performance in higher average price transactions.

**Recommendations:**
* Assign them premium/luxury lines of products.
* Involve them in high-value clients acquisiton strategies.
* Watch and replicate their personal strategies across the team.

**Rationale:**
Margin optmization and right talent placement may present stronger profitability than pure volume increase.

## 8.3 Review Underperforming Sales Positions
The lowest-performing sales representatives show noticeable revenue gaps compared to the team average.

**Recommendations:**
* Provide personal coaching.
* Evaluate geographical location.

**Rationale:**
Improving the table bottom performers can lead to significant revenue effect.

## 8.4 Evaluate Product Pricing Strategy
Products such as Caramel Stuffed Bars show strong volume but low average price.

**Recommendations:**
* Test moderate pricing changes.
* New lines maintaning the base flavour but exploring diverse additionals with higher price.

**Rationale:**
Small margin increases on high-volume products can generate a substantial revenue rise.

## 8.5 Analyze Seasonal Volume Patterns
January and June revenue peaks are driven by shipment volume.

**Recommendations:**
* Prepare inventory scaling strategies during peak months.
* Launch targeted promotions during historically strong demand periods.
* Investigate demand drivers for these months.

**Rationale:**
Understading volume peaks and seasonal changes can improve the operation flow and arrangements.

# 9. Limitations & Future Improvements
## 9.1 Dataset Scope
The dataset consists of a single transactional table and does not include cost data, profit margins, or operational expenses.

**Impact:**

While revenue performance could be analyzed, profitability and margin efficiency could not be evaluated. Revenue growth does not necessarily reflect profit growth.
## 9.2 Learning-Oriented Dataset Structure
As the dataset is designed for learning purposes, it presented minimal structural inconsistencies, spelling errors, formatting issues, or null values.

**Impact:**

Although this alleviated preprocessing steps, it also limited experience with real-world data quality challenges such as inconsistent encoding, missing values, or messy formatting normally found in operational datasets. This suggests that future projects should incorporate more complex or raw datasets to further develop data cleaning mastery.
## 9.3 Limited Temporal & Geographic Scope
The dataset covers a limited time period and six countries.

**Impact:**

This impacts the ability to perform deeper trend analysis, long-term planning, or regional behaviour studies. Broader time spanning and additional marked would strengthen data-based conclusions.
## 9.4 Absence of Customer-Level Data
No customer IDs or repeat purchase metrics were available.

**Impact:**

Customer retention, segmentation, and lifetime value analysis could not be proceeded.
## 9.5 Seasonal Interpretation Constraints
Monthly revenue patterns suggest possible seasonal effects; however, no external marketing, economic, or event data were available to empower such hypothesis.

**Impact:**

Seasonal interpretations remain observational guesses and could not be consolidated.